In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os import path
from deap import creator, base, tools, gp
import random
import csv
import operator
import math
random.seed(117)

In [2]:
# PATH = 'grape/'

# # check if 'grape' already exists
# if os.path.exists(PATH):
#     print('grape directory already exists')

# # change directory to 'grape'
# %cd grape/

In [3]:
train_file = 'train.csv'
# load train set
df_train = pd.read_csv(train_file)
df_train.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,output
0,0.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,7.0,6.0,8.0,0.0
1,1.0,1.0,1.0,35.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,9.0,6.0,8.0,0.0
2,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,1.0,0.0,13.0,5.0,4.0,0.0
3,0.0,0.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,7.0,4.0,6.0,0.0
4,1.0,0.0,1.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,3.0,0.0,0.0,0.0,8.0,5.0,7.0,0.0


In [4]:
df_train.describe()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,output
count,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,...,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000
mean,0.513883,0.493653,0.972035,29.447441,0.473225,0.050377,0.133082,0.728877,0.616025,0.801666,...,0.085879,2.702102,3.617017,5.195954,0.207061,0.447441,8.376041,4.958548,5.852836,0.356010
std,0.499857,0.500009,0.164889,7.080019,0.499332,0.218743,0.339697,0.444583,0.486400,0.398785,...,0.280213,1.106379,7.997166,9.538762,0.405240,0.497279,2.920253,1.018217,2.131317,0.478866
min,0.000000,0.000000,0.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000,4.000000,0.000000
50%,1.000000,0.000000,1.000000,28.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,9.000000,5.000000,6.000000,0.000000
75%,1.000000,1.000000,1.000000,33.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,3.000000,2.000000,5.000000,0.000000,1.000000,10.000000,6.000000,8.000000,1.000000
max,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,5.000000,30.000000,30.000000,1.000000,1.000000,13.000000,6.000000,8.000000,1.000000


In [5]:
X_train = df_train.copy()
# warning: cannot drop it more than once
X_train.drop(['output'], axis=1, inplace=True)

y_train = df_train['output'].to_numpy()

In [6]:
print(y_train[0:5]) #print head

[0. 0. 0. 0. 0.]


In [7]:
test_file = 'test.csv'
# load test set
df_test = pd.read_csv(test_file)
df_test.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,28.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,4.0,2.0,2.0,1.0,1.0,8.0,4.0,1.0
1,1.0,0.0,1.0,27.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,14.0,0.0,0.0,11.0,6.0,6.0
2,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,3.0,0.0,0.0,1.0,12.0,6.0,6.0
3,0.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,6.0,3.0
4,1.0,1.0,1.0,32.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,5.0,3.0,6.0,1.0,0.0,10.0,5.0,6.0


In [8]:
df_test.describe()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
count,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,...,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000,95804.000000
mean,0.508330,0.484239,0.970628,29.262797,0.461442,0.053839,0.125329,0.724427,0.621352,0.798161,...,0.953353,0.088973,2.701891,3.488654,5.155662,0.219573,0.447841,8.362761,4.977642,5.846739
std,0.499933,0.499754,0.168849,6.972092,0.498514,0.225701,0.331093,0.446805,0.485053,0.401375,...,0.210883,0.284707,1.106275,7.819002,9.550624,0.413960,0.497275,2.959191,1.012794,2.136366
min,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000,4.000000
50%,1.000000,0.000000,1.000000,28.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,9.000000,5.000000,6.000000
75%,1.000000,1.000000,1.000000,32.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,3.000000,2.000000,5.000000,0.000000,1.000000,11.000000,6.000000,8.000000
max,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,30.000000,30.000000,1.000000,1.000000,13.000000,6.000000,8.000000


In [9]:
X_test = df_test.copy()

In [10]:
Binary_data = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']
Categorical_data = ['GenHlth', 'Age', 'Education', 'Income']
Numerical_data = ['BMI', 'MentHlth', 'PhysHlth','MentHlth', 'PhysHlth']

In [11]:
X_train[Categorical_data]

,GenHlth,Age,Education,Income
0,1.0,7.0,6.0,8.0
1,2.0,9.0,6.0,8.0
2,2.0,13.0,5.0,4.0
3,2.0,7.0,4.0,6.0
4,3.0,8.0,5.0,7.0
...,...,...,...,...
5037,3.0,12.0,4.0,3.0
5038,2.0,8.0,5.0,8.0
5039,3.0,11.0,4.0,2.0
5040,4.0,13.0,5.0,4.0


In [12]:
from sklearn.preprocessing import OneHotEncoder # type: ignore
drop_enc = OneHotEncoder()
drop_enc.fit(df_train[Categorical_data])
X_train_cat = drop_enc.transform(df_train[Categorical_data])

In [13]:
X_train_cat = X_train_cat.toarray()
X_test_cat = drop_enc.transform(df_test[Categorical_data])
X_test_cat = X_test_cat.toarray()

In [14]:
X_train = X_train[Binary_data].join(pd.DataFrame(X_train_cat)) 
X_test = X_test[Binary_data].join(pd.DataFrame(X_test_cat))
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [15]:
X_train.shape

(5042, 46)

In [16]:
# Define the function set
def and_int(x1, x2):
    if x1 == x2:
        return 1
    else:
        return 0
def or_int(x1, x2):
    if x1 == 1 or x2 == 1:
        return 1
    else:
        return 0

In [17]:
# Create a primitive set for GP
pset = gp.PrimitiveSet("MAIN", X_train.shape[1])  # Each feature in X is a variable
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(and_int, 2)
pset.addPrimitive(or_int, 2)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.addTerminal(1)
pset.addTerminal(0)

# Define Fitness and Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

/home/achyut/micromamba/envs/UL/lib/python3.10/site-packages/deap/gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [18]:
from sklearn.metrics import accuracy_score

# Define the evaluation function
def eval_individual(individual):
    func = toolbox.compile(expr=individual)
    predictions = [func(*x) > 0 for x in X_train]  # Binary classification: >0 as class 1
    size_penalty = len(individual) * 0.0000001
    return accuracy_score(y_train, predictions) - size_penalty,
    # return accuracy_score(y_train, predictions),

toolbox.register("evaluate", eval_individual)
toolbox.register("select", tools.selTournament, tournsize=5)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)


In [ ]:
# Parameters
population = toolbox.population(n=500)
num_generations = 200
cxpb, mutpb = 0.5, 0.2  # Crossover and mutation probabilities
elite_size = 5  # Number of best individuals to carry over to the next generation

# Statistics to track progress
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Run the GP evolution
for gen in range(num_generations):
    # Adaptive crossover and mutation
    # cxpb = max(0.3, 0.8 - 0.02 * gen)  # Adaptive crossover rate
    # mutpb = max(0.1, 0.5 - 0.01 * gen)  # Adaptive mutation rate

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate fitness of individuals with invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    elite = tools.selBest(population, elite_size)

    # Replace old population with offspring
    population[:] = offspring
    # population[:] = elite + offspring[:500 - elite_size]

    # Gather and print statistics for the generation
    record = stats.compile(population)
    print(f"Generation {gen}: {record}")

# Find the best individual
best_individual = tools.selBest(population, 1)[0]
print("Best individual:", best_individual)
print("Best individual fitness:", best_individual.fitness.values)


Generation 0: {'avg': 0.559159259999524, 'std': 0.10441899030991038, 'min': 0.3343908146370488, 'max': 0.6943670145577152}
Generation 1: {'avg': 0.6283659458217374, 'std': 0.061340740858417016, 'min': 0.317929093097977, 'max': 0.6943670145577152}
Generation 2: {'avg': 0.6435487973646172, 'std': 0.05976103899313912, 'min': 0.32149780499801667, 'max': 0.6943670145577152}
Generation 3: {'avg': 0.6543382136625149, 'std': 0.05846376572289439, 'min': 0.32645695485918286, 'max': 0.6955570185243951}
Generation 4: {'avg': 0.6586639670813963, 'std': 0.06313706715237219, 'min': 0.34787722625942086, 'max': 0.6955570185243951}
Generation 5: {'avg': 0.6629329481778659, 'std': 0.0677471138547519, 'min': 0.350059100198334, 'max': 0.6955570185243951}
Generation 6: {'avg': 0.6703610029378817, 'std': 0.060625898913213766, 'min': 0.3343896146370488, 'max': 0.6955570185243951}
Generation 7: {'avg': 0.6691801195349465, 'std': 0.06517175599242418, 'min': 0.3391506305037683, 'max': 0.6955570185243951}
Generat

In [ ]:
best_func = toolbox.compile(expr=best_individual)
predictions = [best_func(*x) > 0 for x in X_test] # Binary classification: >0 as class 1

NameError: name 'best_individual' is not defined

In [ ]:
predictions = [int(x) for x in predictions]
predictions[:10]

[1, 1, 0, 0, 1, 0, 0, 0, 1, 0]

In [ ]:
with open('output.csv', 'w') as f:
    f.write('index,output\n')
    for i in enumerate(predictions):
        f.write(f"{i[0]},{i[1]}\n")